# StroopActFlow
### Identify stimuli localizers for input layer for Stroop Model
### Identify top-down task rule inputs (e.g., PFC) for Stroop Model


#### Taku Ito
#### 1/25/2017


In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import statsmodels.sandbox.stats.multicomp as mc
import os
import nibabel as nib
from sklearn.svm import SVC

In [2]:
def convertCSVToCIFTI64k(inputfilename,outputfilename):
    ciftitemplate = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii'
    wb_command = 'wb_command -cifti-convert -from-text' 
    wb_command += ' ' + inputfilename 
    wb_command += ' ' + ciftitemplate
    wb_command += ' ' + outputfilename
    wb_command += " -col-delim ','"
    wb_command += ' -reset-scalars'
    os.system(wb_command)
#     print wb_command

## 0.0 Set up

In [8]:
subjNums = ['101', '102']

basedir = '/projects3/StroopActFlow/data/'
resultsdir = basedir + 'results/'
restdir = resultsdir + 'glmRest_GlasserParcels/'

glasser_nets = np.loadtxt('/projects/AnalysisTools/netpartitions/ColeLabNetPartition_v1.1/community_order.txt', delimiter=',')
# Make into python numbering (starts from 0)
glasser_nets -= 1.0
networkorder = glasser_nets.astype(int)
networkorder.shape = (len(networkorder),1)

networkmappings = {'fpn':7, 'vis':1, 'smn':2, 'con':3, 'dmn':6, 'aud1':8, 'aud2':9, 'dan':11}
networks = networkmappings.keys()

networkdef = '/projects/AnalysisTools/netpartitions/ColeLabNetPartition_v1.1/parcel_network_assignments.txt'
networkdef = np.loadtxt(networkdef, delimiter=',')
xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net
    
# Load in Glasser parcels
glasserfile = '/projects3/StroopActFlow/data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii'
glasser = nib.load(glasserfile).get_data()
glasser = np.squeeze(glasser)

## 2.0 Run task activation analysis on 64k Surface for four different stimuli localizers

In [17]:
def loadBetas(subj):
    datadir = '/projects3/StroopActFlow/data/results/glm_neutral_stimuli/'
    # Load smoothed betas!
    betas = np.loadtxt(datadir + subj + '_neutralStimuliByTrial_taskBetas_Surface64k.csv',delimiter=',')
    betas = betas[:,18:] # 18 onwards are the two task betas
    return betas

## 2.1 Run 2 MVPA analysis 
### Color Localizer: Green V Red
### Word Localizer: Green V Red
* Stim 1: Neutral Color Green
* Stim 2: Neutral Color Red
* Stim 3: Neutral Word Green
* Stim 4: Neutral Word Red

##### Load in data as dicts

In [79]:
ntrials_perstim = 45
nstims = 4
colorstims = [0,1] # First 2 are color
wordstims = [2,3] # second 2 stims are word

beta_dict = {}

# Organize subject betas by stimulus
for subj in subjNums:
    print 'Loading betas for subject', subj
    betas = loadBetas(subj)
    i=0
    beta_dict[subj] = np.zeros((betas.shape[0],ntrials_perstim,nstims))
    for stim in range(nstims):
        for trial in range(ntrials_perstim):
            beta_dict[subj][:,trial,stim] = betas[:,i]
            i += 1
            
        
        

Loading betas for subject 101
Loading betas for subject 102


##### Run 2 classifications

Function for cross-validation

In [184]:
def runCV(svmmat, labels, cvs=10, leaveout=5, replacement=False):
    """
    Runs a cross validation given an svm matrix, labels, number of cross validations, and 
    number of leave outs for test predictions
    
    Assumes cross validation without replacement (for each train v. test prediction)
    """
    # Get number of classes
    classes = np.unique(labels)
    
    # Run cross-validation using Linear SVMs
    leaveout_total = leaveout*len(classes)

    # Spatially demean (across features, to ensure that mean activity does not factor into classification)
    spatialmean = np.mean(svmmat,axis=1)
    spatialmean.shape = (len(spatialmean),1)
    svmmat = svmmat - spatialmean
    
    # Start cross-validation
    nsamples = svmmat.shape[0]
    acc = []
    for cv in range(cvs):
        test_ind = []
        for stim in classes:
            stimlabs = np.where(labels==stim)[0]
            test_ind.extend(np.random.choice(stimlabs,leaveout,replace=replacement))
            
        test_ind = np.asarray(test_ind)
        train_ind = np.delete(np.arange(nsamples),test_ind)

        trainset = svmmat[train_ind,:]
        testset = svmmat[test_ind,:]
        # normalize train and test set according to train set mean & std
        mean = np.mean(trainset,axis=0)
        std = np.std(trainset,axis=0)
        trainset = np.divide((trainset - mean),std)
        testset = np.divide((testset - mean),std)

        # Construct classifier and fit
        svc = SVC(kernel='linear')
        svc.fit(trainset,labels[train_ind])
        # Get acc
        acc.append(svc.score(testset,labels[test_ind]))

    # Get average accuracy
#     accmat[subj][roi_ind,0] = np.mean(acc)
    avg_acc = np.mean(acc)

    
    # Perform within-subject binomial test
    ntotal = leaveout_total*cvs
    nsuccess = avg_acc*ntotal
    chance = 1.0/len(classes)
    p = stats.binom_test(nsuccess,ntotal,p=chance)

    # Make sure it's a one-sided binomial test
    if avg_acc > chance:
        p = p/2.0
    else:
        p = 1.0 - p/2.0
    
    return avg_acc, p


#### Run actual code/cross-validations

In [185]:
accmat = {}
pmat = {}
qmat = {}
accmat_thresh = {}
nparcels = 360

for subj in subjNums:
    print 'Running classifications for subject', subj
    accmat[subj] = np.zeros((glasser.shape[0],3)) # 2 for two classifications; 3rd is for 4-way class    
    pmat[subj] = np.zeros((nparcels,3))
    
    for roi in range(nparcels):
        if nparcels%100==0:
            print 'Running classifications on ROI,', roi + 1
        roi_ind = np.where(glasser==roi+1)[0]
        
        ####
        ## Color STIM classification        
        # Set up SVM Matrix
        mat_tmp = [] # For a 2-way classification
        labels_tmp = [] 
        for stim in colorstims:
            mat_tmp.append(beta_dict[subj][roi_ind,:,stim].T)
            labels_tmp.append(np.ones((ntrials_perstim,1))*stim)
        svmmat_color = np.vstack(mat_tmp)
        labels_color = np.squeeze(np.vstack(labels_tmp))
        
        # Run cross-validation using Linear SVMs
        crossvalidations = 10 # 10-fold cross validation; 90 samples, 81 train; 9 test
        leaveout = 5 # leave out 5 samples of each stim

        acc, p = runCV(svmmat_color, labels_color, cvs=crossvalidations, leaveout=leaveout, replacement=False)
        accmat[subj][roi_ind,0] = acc
        pmat[subj][roi,0] = p
        h0, qs = mc.fdrcorrection0(pmat[subj][:,0])
        
        
        ####
        ## Word STIM classification        
        # Set up SVM Matrix
        mat_tmp = [] # For a 2-way classification
        labels_tmp = [] 
        for stim in wordstims:
            mat_tmp.append(beta_dict[subj][roi_ind,:,stim].T)
            labels_tmp.append(np.ones((ntrials_perstim,1))*stim)
        svmmat_word = np.vstack(mat_tmp)
        labels_word = np.squeeze(np.vstack(labels_tmp))
        
        # Run cross-validation using Linear SVMs
        crossvalidations = 10 # 10-fold cross validation; 90 samples, 81 train; 9 test
        leaveout = 5 # leave out 5 samples of each stim

        acc, p = runCV(svmmat_word, labels_word, cvs=crossvalidations, leaveout=leaveout, replacement=False)
        accmat[subj][roi_ind,1] = acc
        pmat[subj][roi,1] = p
        
        
        ####
        ## 4-way STIM classification        
        # Set up SVM Matrix
        svmmat_4way = np.vstack((svmmat_color,svmmat_word))
        labels_4way = np.squeeze(np.hstack((labels_color,labels_word)))
        
        # Run cross-validation using Linear SVMs
        crossvalidations = 10 # 10-fold cross validation; 90 samples, 81 train; 9 test
        leaveout = 5 # leave out 5 samples of each stim

        acc, p = runCV(svmmat_4way, labels_4way, cvs=crossvalidations, leaveout=leaveout, replacement=False)
        accmat[subj][roi_ind,2] = acc
        pmat[subj][roi,2] = p
        
        
        
        

Running classifications for subject 101
Running classifications for subject 102


In [196]:
print accmat['101'][roi_ind,2]
print accmat['102'][roi_ind,1]

[ 0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34
  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34
  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34
  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34
  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34  0.34]
[ 0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55
  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55
  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55
  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55
  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55  0.55]


#### Run FDR correction for each test

In [186]:
qmat = {}
accmat_thresh = {}

for subj in subjNums:
    accmat_thresh[subj] = np.zeros(accmat[subj].shape)
    
    for test in range(accmat[subj].shape[1]):
        h0,qs = mc.fdrcorrection0(pmat[subj][:,test])
        
        # Iterate through each ROI and make sure it is significant
        for roi in range(nparcels):
            roi_ind = np.where(glasser==roi+1)[0]
            if qs[roi] < 0.05:
                accmat_thresh[subj][roi_ind,test] = accmat[subj][roi_ind,test]

#### Save to CSV

In [187]:
basedir = '/projects3/StroopActFlow/data/results/glm_neutral_stimuli/'
for subj in subjNums:
    
    filename_mvpa = basedir + subj + '_neutralStimuliMVPA_Acc.csv'
    filename_mvpafdr = basedir + subj + '_neutralStimuliMVPA_AccThresh.csv'
        
    np.savetxt(filename_mvpa, accmat[subj], delimiter=',')
    np.savetxt(filename_mvpafdr, accmat_thresh[subj], delimiter=',')

#### Convert statistical maps to surface maps

In [197]:
basedir = '/projects3/StroopActFlow/data/results/glm_neutral_stimuli/'
for subj in subjNums:

    
    cifti_mvpa = basedir + subj + '_neutralStimuliMVPA_Acc.dscalar.nii'
    cifti_mvpafdr = basedir + subj + '_neutralStimuliMVPA_AccThresh.dscalar.nii'
    
    filename_mvpa = basedir + subj + '_neutralStimuliMVPA_Acc.csv'
    filename_mvpafdr = basedir + subj + '_neutralStimuliMVPA_AccThresh.csv'
    
    convertCSVToCIFTI64k(filename_mvpa, cifti_mvpa)
    convertCSVToCIFTI64k(filename_mvpafdr, cifti_mvpafdr)

## 2.2 Run statistical analysis (i.e., activation threshold analysis) with contrasts against other stimulus conditions

In [150]:
ntrials_perstim = 45
nstims = 4

tmat = {}
pmat = {}
qmat = {}
tmat_fdr = {}

for subj in subjNums:
    print 'Running within-subject analysis on subj', subj
    
    betas = loadBetasFromDscalar(subj)
    betas = stats.zscore(betas,axis=0)    
    tmat[subj] = np.zeros((betas.shape[0],nstims))
    pmat[subj] = np.zeros((betas.shape[0],nstims))
    qmat[subj] = np.zeros((betas.shape[0],nstims))
    tmat_fdr[subj] = np.zeros((betas.shape[0],nstims))
    
    # Construct index for which stims corresopnd to which columns
    stim_ind = np.arange(4,dtype=np.ndarray)
    stim_ind.shape = (stim_ind.shape[0],1)
    stim_ind = np.reshape(np.tile(a,ntrials_perstim),-1)
    for stim in range(nstims):
        print 'Thresholding statistical maps for stim', stim
        beta_mat = np.zeros((betas.shape[0],ntrials_perstim))
        beta_mat_contrast = np.zeros((betas.shape[0],ntrials_perstim*(nstims-1)))
                          
        cond_ind = np.where(stim_ind==stim)[0]
        beta_mat[:,:] = betas[:,cond_ind]
        
        notcond_ind = np.where(stim_ind!=stim)[0]
        beta_mat_contrast[:,:] = betas[:,notcond_ind]        
        
        # Run t-test
        t = []
        p = []
        for vertex in range(beta_mat.shape[0]):
            t, p = stats.ttest_ind(beta_mat[vertex,:],beta_mat_contrast[vertex,:])
            
            # Make one-sided t-test
            if t > 0:
                p = p/2.0
            else:
                p = 1.0 - p/2.0
                
            tmat[subj][vertex,stim] = t
            pmat[subj][vertex,stim] = p
            

            
        # Run FDR correction
        h0, qmat[subj][:,stim] = mc.fdrcorrection0(pmat[subj][:,stim])
        # Threshold tmat for significance
        tmat_fdr[subj][:,stim] = np.multiply(h0,tmat[subj][:,stim]) 
        
        

Running within-subject analysis on subj 101
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1
Thresholding statistical maps for stim 2
Thresholding statistical maps for stim 3
Running within-subject analysis on subj 102
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1
Thresholding statistical maps for stim 2
Thresholding statistical maps for stim 3


#### Save to CSV

In [151]:
basedir = '/projects3/StroopActFlow/data/results/glm_neutral_stimuli/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_neutralStimuliByTrial_tstats_StimVAll.csv'
    filename_tmatfdr = basedir + subj + '_neutralStimuliByTrial_tstatsThresh_StimVAll.csv'
        
    np.savetxt(filename_tmat, tmat[subj], delimiter=',')
    np.savetxt(filename_tmatfdr, tmat_fdr[subj], delimiter=',')

## Convert statistical maps to surface maps

In [153]:
basedir = '/projects3/StroopActFlow/data/results/glm_neutral_stimuli/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_neutralStimuliByTrial_tstats_StimVAll.csv'
    filename_tmatfdr = basedir + subj + '_neutralStimuliByTrial_tstatsThresh_StimVAll.csv'
    
    cifti_tmat = basedir + subj + '_neutralStimuliByTrial_tstats_StimVAll.dscalar.nii'
    cifti_tmatfdr = basedir + subj + '_neutralStimuliByTrial_tstatsThresh_StimVAll.dscalar.nii'
    
    convertCSVToCIFTI64k(filename_tmat, cifti_tmat)
    convertCSVToCIFTI64k(filename_tmatfdr, cifti_tmatfdr)

___

## 3.0 Run task-rule encoding against 0
### Isolate top-down control (i.e., hypothesized PFC)

In [157]:
def loadBetasFromDscalarTaskRuleEncoding(subj):
    datadir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/'
    # Load smoothed betas!
    betas = nib.load(datadir + subj + '_taskRuleEncoding_taskBetas_Surface64k_sm.dscalar.nii')
    betas = betas.get_data()
    betas = np.squeeze(betas)
    betas = betas.T
    betas = betas[:,18:] # 18 onwards are the two task betas
    return betas

In [167]:
for subj in subjNums:
    print 'Subject', subj
    glmdir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/' 
    
    print 'Converting CSV to dscalar...'
    csvfile = glmdir + subj + '_taskruleEncoding_taskBetas_Surface64k.csv'
    dscalarfile = glmdir + subj + '_taskRuleEncoding_taskBetas_Surface64k.dscalar.nii'
    convertCSVToCIFTI64k(csvfile, dscalarfile)
    
    print 'Smoothing surface files...'
    dscalar_sm_file = glmdir + subj + '_taskRuleEncoding_taskBetas_Surface64k_sm.dscalar.nii'
    subjdir = '/projects3/StroopActFlow/data/' + subj + '/MNINonLinear/fsaverage_LR32k/'
    lsurf = subjdir + subj + '.L.inflated.32k_fs_LR.surf.gii'
    rsurf = subjdir + subj + '.R.inflated.32k_fs_LR.surf.gii'
    
    wb_command = 'wb_command -cifti-smoothing ' + dscalarfile + ' 4 4 COLUMN ' + dscalar_sm_file
    wb_command += ' -left-surface ' + lsurf
    wb_command += ' -right-surface ' + rsurf
    os.system(wb_command)

Subject 101
Converting CSV to dscalar...
Smoothing surface files...
Subject 102
Converting CSV to dscalar...
Smoothing surface files...


### 3.0.1 Run statistical analysis (i.e., activation threshold analysis) against 0
* Task 1: ColorEncoding
* Task 2: WordEncoding

Betas are z-scored prior to running t-tests against 0

In [191]:
ntrials_perstim = 30
nstims = 2

tmat = {}
pmat = {}
qmat = {}
tmat_fdr = {}

for subj in subjNums:
    print 'Running within-subject analysis on subj', subj
    
    betas = loadBetasFromDscalarTaskRuleEncoding(subj)
    betas = stats.zscore(betas,axis=0)
    tmat[subj] = np.zeros((betas.shape[0],nstims))
    pmat[subj] = np.zeros((betas.shape[0],nstims))
    qmat[subj] = np.zeros((betas.shape[0],nstims))
    tmat_fdr[subj] = np.zeros((betas.shape[0],nstims + 1)) # Include conjunction
    
    # Construct index for which stims corresopnd to which columns
    stim_ind = np.arange(nstims,dtype=np.ndarray)
    stim_ind.shape = (stim_ind.shape[0],1)
    stim_ind = np.reshape(np.tile(stim_ind,ntrials_perstim),-1)
    for stim in range(nstims):
        print 'Thresholding statistical maps for stim', stim
        beta_mat = np.zeros((betas.shape[0],ntrials_perstim))
                          
        cond_ind = np.where(stim_ind==stim)[0]
        beta_mat[:,:] = betas[:,cond_ind]
            
        # Run t-test
        t = []
        p = []
        for vertex in range(beta_mat.shape[0]):
            t, p = stats.ttest_1samp(beta_mat[vertex,:],0)
            
            # Make one-sided t-test
            if t > 0:
                p = p/2.0
            else:
                p = 1.0 - p/2.0
                
            tmat[subj][vertex,stim] = t
            pmat[subj][vertex,stim] = p
            

            
        # Run FDR correction
        h0, qmat[subj][:,stim] = mc.fdrcorrection0(pmat[subj][:,stim])
        # Threshold tmat for significance
        tmat_fdr[subj][:,stim] = np.multiply(h0,tmat[subj][:,stim])         
        # Include conjunction map
        tmat_fdr[subj][:,2] = tmat_fdr[subj][:,2] + h0
        
    tmat_fdr[subj][:,2] = tmat_fdr[subj][:,2] == nstims

Running within-subject analysis on subj 101
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1
Running within-subject analysis on subj 102
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1


#### Save to CSV

In [192]:
basedir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_taskRuleEncoding_tstats.csv'
    filename_tmatfdr = basedir + subj + '_taskRuleEncoding_tstatsThresh.csv'
        
    np.savetxt(filename_tmat, tmat[subj], delimiter=',')
    np.savetxt(filename_tmatfdr, tmat_fdr[subj], delimiter=',')

#### Convert statistical maps to surface maps

In [193]:
basedir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_taskRuleEncoding_tstats.csv'
    filename_tmatfdr = basedir + subj + '_taskRuleEncoding_tstatsThresh.csv'
    
    cifti_tmat = basedir + subj + '_taskRuleEncoding_tstats.dscalar.nii'
    cifti_tmatfdr = basedir + subj + '_taskRuleEncoding_tstatsThresh.dscalar.nii'
    
    convertCSVToCIFTI64k(filename_tmat, cifti_tmat)
    convertCSVToCIFTI64k(filename_tmatfdr, cifti_tmatfdr)

## 3.1 Run task-rule encoding contrasts (color v. word)
### Isolate top-down control (i.e., hypothesized PFC)

### 3.1.1 Run statistical analysis (i.e., activation threshold analysis) against 0
* Task 1: ColorEncoding
* Task 2: WordEncoding

Betas are z-scored prior to running t-tests against 0

In [186]:
ntrials_perstim = 30
nstims = 2

tmat = {}
pmat = {}
qmat = {}
tmat_fdr = {}

for subj in subjNums:
    print 'Running within-subject analysis on subj', subj
    
    betas = loadBetasFromDscalarTaskRuleEncoding(subj)
    betas = stats.zscore(betas,axis=0)
    tmat[subj] = np.zeros((betas.shape[0],nstims))
    pmat[subj] = np.zeros((betas.shape[0],nstims))
    qmat[subj] = np.zeros((betas.shape[0],nstims))
    tmat_fdr[subj] = np.zeros((betas.shape[0],nstims)) # Include conjunction
    
    # Construct index for which stims corresopnd to which columns
    stim_ind = np.arange(nstims,dtype=np.ndarray)
    stim_ind.shape = (stim_ind.shape[0],1)
    stim_ind = np.reshape(np.tile(stim_ind,ntrials_perstim),-1)
    for stim in range(nstims):
        print 'Thresholding statistical maps for stim', stim
        beta_mat = np.zeros((betas.shape[0],ntrials_perstim))
        beta_mat_contrast = np.zeros((betas.shape[0],ntrials_perstim*(nstims-1)))
                          
        cond_ind = np.where(stim_ind==stim)[0]
        beta_mat[:,:] = betas[:,cond_ind]
      
        notcond_ind = np.where(stim_ind!=stim)[0]
        beta_mat_contrast[:,:] = betas[:,notcond_ind]        
      
        # Run t-test
        t = []
        p = []
        for vertex in range(beta_mat.shape[0]):
            t, p = stats.ttest_rel(beta_mat[vertex,:],beta_mat_contrast[vertex,:])
            
            # Make one-sided t-test
            if t > 0:
                p = p/2.0
            else:
                p = 1.0 - p/2.0
                
            tmat[subj][vertex,stim] = t
            pmat[subj][vertex,stim] = p
            

            
        # Run FDR correction
        h0, qmat[subj][:,stim] = mc.fdrcorrection0(pmat[subj][:,stim])
        # Threshold tmat for significance
        tmat_fdr[subj][:,stim] = np.multiply(h0,tmat[subj][:,stim])         
        

Running within-subject analysis on subj 101
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1
Running within-subject analysis on subj 102
Thresholding statistical maps for stim 0
Thresholding statistical maps for stim 1


#### Save to CSV

In [189]:
basedir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_taskRuleEncoding_ruleContrast_tstats.csv'
    filename_tmatfdr = basedir + subj + '_taskRuleEncoding_ruleContrast_tstatsThresh.csv'
        
    np.savetxt(filename_tmat, tmat[subj], delimiter=',')
    np.savetxt(filename_tmatfdr, tmat_fdr[subj], delimiter=',')

#### Convert statistical maps to surface maps

In [190]:
basedir = '/projects3/StroopActFlow/data/results/glm_taskruleencoding/'
for subj in subjNums:
    
    filename_tmat = basedir + subj + '_taskRuleEncoding_ruleContrast_tstats.csv'
    filename_tmatfdr = basedir + subj + '_taskRuleEncoding_ruleContrast_tstatsThresh.csv'
    
    cifti_tmat = basedir + subj + '_taskRuleEncoding_ruleContrast_tstats.dscalar.nii'
    cifti_tmatfdr = basedir + subj + '_taskRuleEncoding_ruleContrast_tstatsThresh.dscalar.nii'
    
    convertCSVToCIFTI64k(filename_tmat, cifti_tmat)
    convertCSVToCIFTI64k(filename_tmatfdr, cifti_tmatfdr)

### Results 

* No significant voxels for word v. color rule contrasts (within subject)